torch.nn is used to make Neural Networks. nn depends on autograd to define models and differentiate them.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1,6,5)
        self.conv2 = nn.Conv2d(6,16,5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120) # 5*5 from image dimension
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, input):
        # Convolution layer C1: 1 input image channel, 6 output channels,
        # 5x5 square convolution, it uses RELU activation function, and
        # outputs a Tensor with size (N, 6, 28, 28), where N is the size of the batch
        c1 = F.relu(self.conv1(input))
        # Subsampling layer S2: 2x2 grid, purely functional,
        # this layer does not have any parameter, and outputs a (N, 6, 14, 14) Tensor
        s2 = F.max_pool2d(c1, (2,2))
        # Convolution layer C3: 6 input channels, 16 output channels,
        # 5x5 square convolution, it uses RELU activation function, and
        # outputs a (N, 16, 10, 10) Tensor
        c3 = F.relu(self.conv2(s2))
        # Subsampling layer S4: 2x2 grid, purely functional,
        # this layer does not have any parameter, and outputs a (N, 16, 5, 5) Tensor
        s4 = F.max_pool2d(c3, 2)
        # Flatten operation: purely functional, outputs a (N, 400) Tensor
        s4 = torch.flatten(s4,1)
        # Fully connected layer F5: (N, 400) Tensor input,
        # and outputs a (N, 120) Tensor, it uses RELU activation function
        f5 = F.relu(self.fc1(s4))
        # Fully connected layer F6: (N, 120) Tensor input,
        # and outputs a (N, 84) Tensor, it uses RELU activation function
        f6 = F.relu(self.fc2(f5))
        # Gaussian layer OUTPUT: (N, 84) Tensor input, and
        # outputs a (N, 10) Tensor
        output = self.fc3(f6)
        
        return output

In [3]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [4]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) # this is the weight of conv1
print(params[-1].size()) # weight of output (10 digits)

10
torch.Size([6, 1, 5, 5])
torch.Size([10])


In [5]:
input = torch.randn(1,1,32,32)
out = net(input)
print(out)

tensor([[-0.0943,  0.0069, -0.0827,  0.0196, -0.0142, -0.0788, -0.1564, -0.1251,
          0.0561,  0.0197]], grad_fn=<AddmmBackward0>)


In [6]:
# Zero the gradient buffers of all parameters and backprops with random gradients
net.zero_grad()
out.backward(torch.rand(1,10))

Loss Functions

In [7]:
output = net(input)
target = torch.randn(10) # dummy target
target = target.view(1,-1) # same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.9115, grad_fn=<MseLossBackward0>)


In [8]:
loss.grad

/var/folders/rx/q5w9l9sd7zq8l0t_p73qg_l00000gn/T/ipykernel_29599/3355736690.py:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1716905753263/work/build/aten/src/ATen/core/TensorBody.h:494.)
  loss.grad


In [9]:
# So, when we call loss.backward(), the whole graph is differentiated w.r.t. the neural net parameters, and 
# all Tensors in the graph that have requires_grad=True will have their .grad Tensor accumulated with the gradient.

print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [10]:
# to back propagate the error, we call loss.backward()
# First, clear the existing gradients, to prevent accumulating existing gradients

net.zero_grad()

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None
conv1.bias.grad after backward
tensor([ 0.0045,  0.0013,  0.0050, -0.0287,  0.0224, -0.0210])


Updating the weights of the network

In [11]:
# implementing weight = weight - learning_rate * gradient (Stochastic Gradient Descent)
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [13]:
# torch.optim implements many different update rules, including SGD, Adam, Nesterov-SGD 
import torch.optim as optim

# initialize optimizer
optimizer = optim.SGD(net.parameters(), lr = 0.01)

# training loop:
optimizer.zero_grad() # zeroing the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step() # updates the weights